In [59]:
from dash import Dash, dash_table, html, dcc 
from dash.dependencies import Input, Output
import pandas as pd
import json
from src.employee_keys import extra_column, string_example
from src.setting_keys import extra_column_s


In [60]:
def clean_time(time):
    if(time):
        try: 
            if len(time) == 26: #time with timezone localization
                ntime = time[6:-7]
                ntime = int(ntime) + (3.6e+6)*2 # convert to MET
                ntime = pd.to_datetime(ntime, unit = 'ms').strftime("%d/%m/%y %H:%M")
            if len(time) == 21: #without timezone localization
                print(2)
                ntime = time[6:-2]
                ntime = pd.to_datetime(ntime, unit = 'ms').strftime("%d/%m/%y %H:%M")
            if len(time) == 20:
                print(3)
                ntime = time[6:-2]
                ntime = pd.to_datetime(ntime, unit = 'ms').strftime("%d/%m/%y %H:%M")
        except: 
            ntime = 'Time could not be deciphered. Double check the formatting. Otherwise contact Klara about this'
            
    else: 
        ntime = time
    return ntime

In [61]:
len('/Date(900076640000)/')

20

In [62]:
def make_table_from_json(json_object, key):
    # df = pd.json_normalize(json_object)
    # key_list = list(json_object.keys())
    #key_list.remove('Name')
    if key == 'Employees':
        df = pd.DataFrame(json_object[key])
        
    else: 
        df = pd.DataFrame(json_object, index = [0])
    time_column_names = ['DepartmentStartDate', 'StartDate', 'EndDate', 'ContractTypeStartDate', 'ContractHourStartDate', 'PositionStartDate']
    for column_name in time_column_names:
        if column_name in df.columns: 
            df = df.assign(
                **{column_name: df[column_name].apply(lambda x: clean_time(x))}
            )
    print('step 1')
    df = df.T.reset_index().rename(columns = {0: 'Value', 'index':'Key'})
    print('step 2')
    df['Explanation / Alternate Name'] = df.Key.map(extra_column).fillna('-')
    print('step 3')
    df = df.astype(str)
    print('step 4')
    return df

In [63]:
jsonobject ={
    "Employees": [
	{
        "WageSystemKey": "001",
        "DepartmentKey": "500",
        "WageModel": "Monaslohn",
        "FirstName": "Bob the Builder",
        "StartDate": "/Date(900076640000)/",
        "Email": "bob@building.com",
        "Position": "Assistant",
        "PositionStartDate": "", 
        "ContractHours": "38.5",
        "WorkDaysPerWeek": "5.0",
        "ContractHourStartDate": "/Date(1669852800000)/",
        "HeadCount": "1.0",
        "SetAsHomeDepartment":True,
        "CustomColumns":[
            {"Name":"CriminalRecord","Value":True},
            {"Allergies": ""}
            ] 
    }
],
"ImportSettings":
    {
        "ShouldActivateEmployee":True,
        "ForceUpdateEmployeeHistories":True,
        "ContractTypeStartDateEqualsEmployeeStartDate":True,
        "HomeDepartmentStartDateEqualsEmployeeStartDate":True,
        "PositionStartDateEqualsEmployeeStartDate":True,
        "ShouldUpdateEmptyValuesOnFields":True,
        "ShouldSendActivationEmail":True,
        "MovePreviousEntry":True}
    }

In [64]:
make_table_from_json(jsonobject, 'Employees')

3
2
step 1
step 2
step 3
step 4


,Key,Value,Explanation / Alternate Name
0,WageSystemKey,001,Employee ID as seen in the Interface
1,DepartmentKey,500,Store ID as seen in Department List
2,WageModel,Monaslohn,Contract Type
3,FirstName,Bob the Builder,Actually just the name of the employee
4,StartDate,10/07/98 13:17,Employee Starting Date
5,Email,bob@building.com,-
6,Position,Assistant,-
7,PositionStartDate,,-
8,ContractHours,38.5,-
9,WorkDaysPerWeek,5.0,-
